In [ ]:
import pandas as pd
from pathlib import Path
from io import StringIO
import json

- https://github.com/rafalposwiata/depression-detection-lt-edi-2022

In [ ]:
from transformers import Trainer, TrainingArguments, AutoTokenizer, DataCollatorWithPadding, AutoModelForSequenceClassification
from datasets import load_dataset
import datasets

In [ ]:
import torch
torch.cuda.get_device_name(0)

# col cuore in gola

dataset

In [ ]:
data_text = ""

for filename in sorted(["ds", "ts_hs", "ts_ht"]):
    with open(Path("downloads", "task_0", f"{filename}.tsv"), "rt", encoding="utf8") as f:
        data_text += f.read()

df = pd.read_csv(StringIO(data_text), sep="\t")
# dup_idx = df.index.difference(df.drop_duplicates().index)
# df.loc[dup_idx]
df = df.drop_duplicates().reset_index(names="old_idx").reset_index(names="new_idx")

train test indexes

In [ ]:
lookup = df.set_index("old_idx")["new_idx"]
pth = Path("downloads", "task_0", "train_test_splitting.json")
idx = json.load(open(pth, "rt"))
idx = {k: [lookup[i] for i in lst if i in lookup] for k, lst in idx.items()}


In [ ]:
x, y = df["pp_text"], df["label"]
x_train, x_test = x.loc[idx["train"]], x.loc[idx["test"]]
y_train, y_test = y.loc[idx["train"]], y.loc[idx["test"]]

In [ ]:
dataset = {
    "train": datasets.Dataset.from_list([{"label": int(y), "text": str(x)} for y, x in zip(y_train, x_train)]),
    "test": datasets.Dataset.from_list([{"label": int(y), "text": str(x)} for y, x in zip(y_test, x_test)]),
}

dataset =  datasets.DatasetDict(dataset)

tokens

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("ShreyaR/finetuned-roberta-depression")

In [ ]:
def tokenize_function(examples):
    return tokenizer(examples["text"], padding="max_length", truncation=True)

tokenized_datasets = dataset.map(tokenize_function, batched=True)

In [ ]:
tokenized_datasets.items()

In [ ]:
# data_collator = DataCollatorWithPadding(tokenizer=tokenizer)

finetuning

In [ ]:
model = AutoModelForSequenceClassification.from_pretrained("ShreyaR/finetuned-roberta-depression") # "rafalposwiata/deproberta-large-depression"

In [ ]:
training_args = TrainingArguments(
    "test-trainer",
    # no_cuda=True,
    seed=42,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=8,
    gradient_checkpointing=True,
)


In [ ]:
trainer = Trainer(
    model = model,
    args = training_args,
    train_dataset=tokenized_datasets["train"],
    # eval_dataset=tokenized_datasets["eval"],
    # data_collator=data_collator, 
    # tokenizer=tokenizer,
)

In [ ]:
trainer.train()

In [ ]:
predictions = trainer.predict(tokenized_datasets["test"])

In [ ]:
predictions

metrics

In [ ]:
# from evaluate import load
# squad_metric = load("precision")
# predictions = _
# references = _
# results = squad_metric.compute(predictions=predictions, references=references)
# results

In [ ]:
import evaluate
from datasets import load_dataset
from evaluate import evaluator
from transformers import AutoModelForSequenceClassification, pipeline


task_evaluator = evaluator("text-classification")

eval_results = task_evaluator.compute(
    model_or_pipeline=model,
    data=dataset,
    metric=evaluate.combine(["accuracy", "recall", "precision", "f1"]),
    # label_mapping={"NEGATIVE": 0, "POSITIVE": 1}
)
print(eval_results)